In [1]:
import os
import pandas as pd
import numpy as np
import mygene
import gseapy as gp
import gseapy.plot as gseaplot
import matplotlib.pyplot as plt

from gseapy import enrichment_map
import networkx as nx

# %matplotlib inline
# %config InlineBackend.figure_format='retina' # mac
%load_ext autoreload
%autoreload 2
import pandas as pd
import gseapy as gp
import matplotlib.pyplot as plt

from gseapy import Biomart
bm = Biomart()

from gseapy import gseaplot, heatmap

In [2]:
def enrich_genes(top_ranked_genes):

    enr_Reactome_up = gp.enrichr(gene_list=top_ranked_genes,
                                 gene_sets=['Reactome_2022'],
                                 organism='Human', 
                                 cutoff=0.5)
    
    enr_GOBP_up = gp.enrichr(gene_list=top_ranked_genes,
                             gene_sets=['GO_Biological_Process_2021'],
                             organism='Human', 
                             cutoff=0.5)
    
    enr_GOMF_up = gp.enrichr(gene_list=top_ranked_genes ,
                             gene_sets=['GO_Molecular_Function_2021'],
                             organism='Human', 
                             cutoff=0.5)
    
    
    # Write results to CSV files
    #file_name = os.path.basename(file_path).split('.')[0]
    #enr_Reactome_up.results.to_csv(f'{file_name}_enr_Reactome.csv')
    
    return enr_Reactome_up.results, enr_GOBP_up.results, enr_GOMF_up.results

In [22]:
targets4 = pd.read_csv('targets4.csv')
targets12 = pd.read_csv('targets12.csv')
targets24 = pd.read_csv('targets24.csv')

In [23]:
targets24

,Unnamed: 0,mirna,symbol
0,529,hsa-let-7a-3p,NUDT4
1,530,hsa-let-7a-3p,PRDM16
2,531,hsa-let-7a-3p,BMAL1
3,532,hsa-let-7a-3p,NR2F2
4,533,hsa-let-7a-3p,BTBD3
...,...,...,...
11799,800433,hsa-miR-935,PRKG2
11800,800434,hsa-miR-935,HYCC1
11801,800435,hsa-miR-935,KALRN
11802,800436,hsa-miR-935,PPP4R2


In [24]:
mirnas = set.union(set(targets24['mirna']), set(targets4['mirna']), set(targets24['mirna']))

targets24

,Unnamed: 0,mirna,symbol
0,529,hsa-let-7a-3p,NUDT4
1,530,hsa-let-7a-3p,PRDM16
2,531,hsa-let-7a-3p,BMAL1
3,532,hsa-let-7a-3p,NR2F2
4,533,hsa-let-7a-3p,BTBD3
...,...,...,...
11799,800433,hsa-miR-935,PRKG2
11800,800434,hsa-miR-935,HYCC1
11801,800435,hsa-miR-935,KALRN
11802,800436,hsa-miR-935,PPP4R2


In [25]:
mirnas = list(set(set(targets24['mirna'])))

In [26]:
mirnas[1]

'hsa-miR-4294'

In [27]:
targets24[targets24['mirna'].str.contains(mirnas[0])]

,Unnamed: 0,mirna,symbol
0,529,hsa-let-7a-3p,NUDT4
1,530,hsa-let-7a-3p,PRDM16
2,531,hsa-let-7a-3p,BMAL1
3,532,hsa-let-7a-3p,NR2F2
4,533,hsa-let-7a-3p,BTBD3
...,...,...,...
1489,2018,hsa-let-7a-3p,LSM14A
1490,2019,hsa-let-7a-3p,PPP4R2
1491,2020,hsa-let-7a-3p,GPR37
1492,2021,hsa-let-7a-3p,ERI1


In [28]:
miRNA_targets = {}

# Iterate over each miRNA and find the corresponding targets
for mirna in range(len(mirnas)):
    # Filter the DataFrame based on the miRNA number in the target gene column
                   
    targets = targets24[targets24['mirna'].str.contains(mirnas[mirna])]
    miRNA_targets[f'miR-{mirnas[mirna]}'] = targets

# Create a Pandas Series from the miRNA_targets dictionary
miRNA_targets

{'miR-hsa-let-7a-3p':       Unnamed: 0          mirna  symbol
 0            529  hsa-let-7a-3p   NUDT4
 1            530  hsa-let-7a-3p  PRDM16
 2            531  hsa-let-7a-3p   BMAL1
 3            532  hsa-let-7a-3p   NR2F2
 4            533  hsa-let-7a-3p   BTBD3
 ...          ...            ...     ...
 1489        2018  hsa-let-7a-3p  LSM14A
 1490        2019  hsa-let-7a-3p  PPP4R2
 1491        2020  hsa-let-7a-3p   GPR37
 1492        2021  hsa-let-7a-3p    ERI1
 1493        2022  hsa-let-7a-3p   WDR26
 
 [1494 rows x 3 columns],
 'miR-hsa-miR-4294':        Unnamed: 0         mirna  symbol
 9862       328395  hsa-miR-4294     TFG
 9863       328396  hsa-miR-4294   SRPK2
 9864       328397  hsa-miR-4294   WIPF2
 9865       328398  hsa-miR-4294  FAHD2B
 9866       328399  hsa-miR-4294   PMP22
 ...           ...           ...     ...
 10028      328561  hsa-miR-4294  CTNND1
 10029      328562  hsa-miR-4294  PDZD11
 10030      328563  hsa-miR-4294   PRKG1
 10031      328564  hsa-miR-4

In [29]:
mirnas = list(miRNA_targets.keys())

In [30]:
# Filter out the keys with empty Series values
miRNA_targets_filtered = {k: v for k, v in miRNA_targets.items() if not v.empty}

In [31]:
miRNA_targets_filtered

{'miR-hsa-let-7a-3p':       Unnamed: 0          mirna  symbol
 0            529  hsa-let-7a-3p   NUDT4
 1            530  hsa-let-7a-3p  PRDM16
 2            531  hsa-let-7a-3p   BMAL1
 3            532  hsa-let-7a-3p   NR2F2
 4            533  hsa-let-7a-3p   BTBD3
 ...          ...            ...     ...
 1489        2018  hsa-let-7a-3p  LSM14A
 1490        2019  hsa-let-7a-3p  PPP4R2
 1491        2020  hsa-let-7a-3p   GPR37
 1492        2021  hsa-let-7a-3p    ERI1
 1493        2022  hsa-let-7a-3p   WDR26
 
 [1494 rows x 3 columns],
 'miR-hsa-miR-4294':        Unnamed: 0         mirna  symbol
 9862       328395  hsa-miR-4294     TFG
 9863       328396  hsa-miR-4294   SRPK2
 9864       328397  hsa-miR-4294   WIPF2
 9865       328398  hsa-miR-4294  FAHD2B
 9866       328399  hsa-miR-4294   PMP22
 ...           ...           ...     ...
 10028      328561  hsa-miR-4294  CTNND1
 10029      328562  hsa-miR-4294  PDZD11
 10030      328563  hsa-miR-4294   PRKG1
 10031      328564  hsa-miR-4

In [32]:
enr_Reactome_up, enr_GOBP_up, enr_GOMF_up = enrich_genes(miRNA_targets['miR-hsa-miR-374a-5p']['symbol'].dropna())

curr_mirna = 'miR-hsa-miR-374a-5p'
globals()[f"enr_Reactome_up_{curr_mirna.replace('-', '_')}"] = enr_Reactome_up

In [33]:
miRNA_targets_filtered.keys()

dict_keys(['miR-hsa-let-7a-3p', 'miR-hsa-miR-4294', 'miR-hsa-miR-29b-3p', 'miR-hsa-miR-23b-3p', 'miR-hsa-miR-4296', 'miR-hsa-miR-15b-5p', 'miR-hsa-miR-23a-3p', 'miR-hsa-miR-4295', 'miR-hsa-miR-429', 'miR-hsa-miR-4299', 'miR-hsa-miR-935', 'miR-hsa-miR-155-3p', 'miR-hsa-miR-4290', 'miR-hsa-miR-4292', 'miR-hsa-let-7i-5p', 'miR-hsa-miR-4293', 'miR-hsa-miR-4291', 'miR-hsa-miR-4298', 'miR-hsa-miR-21-5p', 'miR-hsa-miR-4297', 'miR-hsa-miR-374a-5p'])

In [48]:
enr_Reactome_up, enr_GOBP_up, enr_GOMF_up = enrich_genes(miRNA_targets['miR-hsa-miR-29b-3p']['symbol'].dropna())
    
enr_Reactome_up_key = f"enr_Reactome_up_{curr_mirna.replace('-', '_')}"
globals()[enr_Reactome_up_key] = enr_Reactome_up
result_dict = {'miR-hsa-miR-29b-3p': globals()[enr_Reactome_up_key][:20]}

result_dict

{'miR-hsa-miR-29b-3p':          Gene_set                                               Term  Overlap  \
 0   Reactome_2022                   Collagen Formation R-HSA-1474290    28/90   
 1   Reactome_2022         Collagen Chain Trimerization R-HSA-8948216    21/44   
 2   Reactome_2022  Collagen Biosynthesis And Modifying Enzymes R-...    24/67   
 3   Reactome_2022  Assembly Of Collagen Fibrils And Other Multime...    22/57   
 4   Reactome_2022    Extracellular Matrix Organization R-HSA-1474244   35/291   
 5   Reactome_2022     Crosslinking Of Collagen Fibrils R-HSA-2243919     5/10   
 6   Reactome_2022                   Signal Transduction R-HSA-162582  84/2465   
 7   Reactome_2022      Scavenging By Class A Receptors R-HSA-3000480     5/19   
 8   Reactome_2022                     Signaling By PDGF R-HSA-186797     7/52   
 9   Reactome_2022         Defective B3GALTL Causes PpS R-HSA-5083635     6/37   
 10  Reactome_2022                    NCAM1 Interactions R-HSA-419037     6/

In [47]:
enr_Reactome_up, enr_GOBP_up, enr_GOMF_up = enrich_genes(miRNA_targets['miR-hsa-let-7a-3p']['symbol'].dropna())
    
enr_Reactome_up_key = f"enr_Reactome_up_{curr_mirna.replace('-', '_')}"
globals()[enr_Reactome_up_key] = enr_Reactome_up
result_dict = {'miR-hsa-let-7a-3p': globals()[enr_Reactome_up_key][:20]}

result_dict

{'miR-hsa-let-7a-3p':          Gene_set                                               Term  \
 0   Reactome_2022                   Signal Transduction R-HSA-162582   
 1   Reactome_2022  Signaling By Rho GTPases, Miro GTPases And RHO...   
 2   Reactome_2022              Signaling By Rho GTPases R-HSA-194315   
 3   Reactome_2022                       Circadian Clock R-HSA-400253   
 4   Reactome_2022        Gene Expression (Transcription) R-HSA-74160   
 5   Reactome_2022  Signaling By Receptor Tyrosine Kinases R-HSA-9...   
 6   Reactome_2022                       Heme Signaling R-HSA-9707616   
 7   Reactome_2022                     RHO GTPase Cycle R-HSA-9012999   
 8   Reactome_2022  Constitutive Signaling By NOTCH1 t(7;9)(NOTCH1...   
 9   Reactome_2022                  Signaling By NOTCH1 R-HSA-1980143   
 10  Reactome_2022           Vesicle-mediated Transport R-HSA-5653656   
 11  Reactome_2022  SUMO E3 Ligases SUMOylate Target Proteins R-HS...   
 12  Reactome_2022            

In [38]:
miRNA_targets_filtered.keys()

dict_keys(['miR-hsa-let-7a-3p', 'miR-hsa-miR-4294', 'miR-hsa-miR-29b-3p', 'miR-hsa-miR-23b-3p', 'miR-hsa-miR-4296', 'miR-hsa-miR-15b-5p', 'miR-hsa-miR-23a-3p', 'miR-hsa-miR-4295', 'miR-hsa-miR-429', 'miR-hsa-miR-4299', 'miR-hsa-miR-935', 'miR-hsa-miR-155-3p', 'miR-hsa-miR-4290', 'miR-hsa-miR-4292', 'miR-hsa-let-7i-5p', 'miR-hsa-miR-4293', 'miR-hsa-miR-4291', 'miR-hsa-miR-4298', 'miR-hsa-miR-21-5p', 'miR-hsa-miR-4297', 'miR-hsa-miR-374a-5p'])

In [50]:
for curr_mirna in list(miRNA_targets_filtered.keys()):
    # Call the enrich_genes function and store the output in variables with dynamic names
    enr_Reactome_up, enr_GOBP_up, enr_GOMF_up = enrich_genes(miRNA_targets[curr_mirna]['symbol'].dropna())
    
    # Store the enriched gene lists in the globals() dictionary
    enr_Reactome_up_key = f"enr_Reactome_up_{curr_mirna.replace('-', '_')}"
    globals()[enr_Reactome_up_key] = enr_Reactome_up
    result_dict[curr_mirna] = globals()[enr_Reactome_up_key][:20]
    
print(result_dict)

2023-05-12 13:07:45,947 [ERROR] Error fetching enrichment results: Reactome_2022
2023-05-12 13:07:46,239 [ERROR] Error fetching enrichment results: Reactome_2022


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
import pickle

with open('dict_of_dfs.pickle', 'wb') as f:
    pickle.dump(result_dict, f)

In [ ]:
result_dict

In [ ]:
import pandas as pd

# Create an Excel writer object
writer = pd.ExcelWriter('enr_24hrs.xlsx')

# Iterate over the dictionary items
for key, value in result_dict.items():
    # Save each nested table as a separate sheet in the Excel file
    value.to_excel(writer, sheet_name=key, index=False)

# Save the Excel file
writer.save()
